<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Нурлан! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным мометнам (кликабельно)</u></summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
    </br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например: <font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>

<font color='blue' size=3><b>Комментарий ревьюера 2</b></font>

<font color='blue'>Привет еще раз. Спасибо, за исправления. Оформление комментариев по работе сохраняется. Только обозначим, что это вторая итерация.</font>

<font color='blue' size=3><b>Комментарий ревьюера 3</b></font>

<font color='blue'>Привет еще раз. Спасибо, за исправления. Оформление комментариев по работе сохраняется.</font>

## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [118]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
#from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.linalg import DenseVector

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer, StandardScaler  

RANDOM_SEED = 2022    


In [119]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("ML California Housing") \
                    .getOrCreate()

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке. Можно ещё сгрупировать импорты по функциональности и разделить группы пустой строкой.</font>

<font color='blue'><b>Комментарий ревьюера : </b></font> ⚠️\
<font color='darkorange'> Инициировать Spark сессию и загружать данные лучше отдельно от импортов, что бы не запускать этот код если понадобится дополнять или редактировать импорты.</font>

# Подготовка данных

In [120]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)
df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'><s>При использовании .toPandas() ты выгружаешь данные в оперативную память, что при работе с действительно большими данными приведет к переполнению памяти, этот метод применяется только к агрегированным таблицам (либо к ограниченным подвыборкам). Попробуй реализовать схожий функционал с применением методов pySpark.
    </s></font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Всё верно!</font>

In [121]:
print(pd.DataFrame(df.printSchema(),columns=['column', 'type']))

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

Empty DataFrame
Columns: [column, type]
Index: []


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично!  Выведены типы данных.</font>

In [122]:
# чистка данных от пропусков
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [123]:
df_mean = df.fillna(value=0,subset=['total_bedrooms']).select(F.mean(F.col('total_bedrooms')).alias('avg')).collect()
avg = df_mean[0]['avg']
df = df.fillna(value=avg,subset=['total_bedrooms']) # замена NA на среднее значение
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8164523123438,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.2669368821015,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Если изменятся данные, то мы по прежнему будем заполнять константой:( Старайся по возможности заполнять программно.</font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Отлично!</font>

In [124]:
# преобразование колонкок с категориальными значениями техникой One hot encoding
exclude = ['ocean_proximity',"median_house_value"]
numerical_cols = [col for col in df.columns if col not in exclude]
target = "median_house_value" 

# изначально используем StringIndexer
indexer = StringIndexer(inputCol='ocean_proximity', 
                        outputCol='ocean_proximity'+'_idx') 
df = indexer.fit(df).transform(df)

#cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
#df.select(cols).show(3)

#теперь уже и сам OHE
encoder = OneHotEncoder(inputCol='ocean_proximity'+'_idx',outputCol='ocean_proximity'+'_ohe')
df = encoder.fit(df).transform(df)

df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|        

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️ <br>
<font color='green'>Верно, что используешь OHE.</font>

<font color='blue'><b>Комментарий ревьюера 3: </b></font> ⚠️\
<font color='darkorange'> Энкодер тоже лучше обучать только на трейне. При работе моделей будет применятся энкодер обученный на известных предыдущих данных.</font>

In [125]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols,outputCol="numerical_features")
df = numerical_assembler.transform(df)

In [126]:
# разделение выборки
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16418 4222


In [127]:
standardScaler = StandardScaler(inputCol='numerical_features',outputCol="numerical_features_scaled")
scaler = standardScaler.fit(train_data)

In [128]:
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [129]:
print(train_data.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'numerical_features', 'numerical_features_scaled']


In [130]:
all_features = ['ocean_proximity_ohe','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features,
                                  outputCol="features")
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3)

+-------------------+-------------------------+
|ocean_proximity_ohe|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[2],[1.0])|     [-61.952887791441...|
|      (4,[2],[1.0])|     [-61.927977100733...|
|      (4,[2],[1.0])|     [-61.913030686308...|
+-------------------+-------------------------+
only showing top 3 rows



In [131]:
print(test_data.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'numerical_features', 'numerical_features_scaled', 'features']


<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'><s> Стандартизацию и Кодирование нужно проводить после разделения на выборки.</s></font>

<font color='purple'><b>Комментарий студента</b></font>
https://practicum.yandex.ru/learn/data-scientist-plus/courses/985f2976-7f2d-444d-a9e9-ef7fc6a4ff86/sprints/14266/topics/fe73fc6e-8711-4730-8f6b-623fc66822e5/lessons/520bf567-b6cb-46b5-96e2-058e0ac966da/ Согласно лекционного материала Стандартизация и Кодирование происходит до разделения на train и test.

<font color='blue'><b>Комментарий ревьюера 2: </b></font> \
<font color='blue'>Ошибка в обучающих материалах. Если применить кодировку и стандартизацию ко всей выборке, происходит утечка о тестовой части данных в обучающую. Скорее всего, стандартизация и кодировка уже встречались в предыдущих проектах и об этой особенности так же был теоритический материал.\
В Spark, тоже есть конвеер Pipeline. С его помощью можно эту проблему устранить.\
Я написал запрос в контент команду ЯндексПрактикума об этой ошибке.\
Спасибо, что отметил.</font>

<font color='blue'><b>Комментарий ревьюера 3: </b></font> ✔️\
<font color='green'>Супер! Скалер обучен на обучающих данных!</font>

# Обучение моделей

In [133]:
#обучение со всеми данными
lr = LinearRegression(labelCol=target, featuresCol='features')

model = lr.fit(train_data)

22/10/17 19:12:51 WARN Instrumentation: [b7aadc6b] regParam is zero, which might cause numerical instability and overfitting.
22/10/17 19:12:51 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/10/17 19:12:51 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/10/17 19:12:51 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/10/17 19:12:51 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
22/10/17 19:12:51 WARN Instrumentation: [b7aadc6b] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [134]:
#обучение с количественными данными
lr2 = LinearRegression(labelCol=target, featuresCol='numerical_features')

model2 = lr2.fit(train_data)

22/10/17 19:12:57 WARN Instrumentation: [00fb8c84] regParam is zero, which might cause numerical instability and overfitting.


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Верно, обучены две модели с категориальным признаком и без.</font>

# Анализ результатов

In [135]:
predictions = model.transform(test_data)

predictedLabes = predictions.select("median_house_value", "prediction")
predictedLabes.show() 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|152871.37074076058|
|           50800.0|214880.87368393922|
|           58100.0|142623.26661453675|
|           68400.0|132355.00961040286|
|           72200.0| 163944.3725004634|
|           67000.0|154339.77777568484|
|           81300.0|152713.30468632653|
|           70500.0| 164411.1542104492|
|           60000.0|142665.50904478086|
|          109400.0|171165.97153167473|
|           74100.0|150527.01439053146|
|           74700.0|167731.97804803262|
|           90000.0|209696.85186513187|
|          104200.0|200353.50041028718|
|           74100.0| 157003.2677775384|
|           67500.0|148170.52536961343|
|          103100.0|47320.383557249326|
|           92500.0|  166603.000242671|
|          128100.0|221965.17673258623|
|           99600.0|187134.27064319467|
+------------------+------------------+
only showing top 20 rows



In [136]:
predictions2 = model2.transform(test_data)

predictedLabes2 = predictions2.select("median_house_value", "prediction")
predictedLabes2.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|101364.50812742068|
|           50800.0|183297.65642193705|
|           58100.0|109578.60838986281|
|           68400.0| 80391.59191420209|
|           72200.0|129877.01967613352|
|           67000.0|120404.20931970468|
|           81300.0|117992.55539450096|
|           70500.0|130612.12188887084|
|           60000.0|110062.61268041749|
|          109400.0| 118172.5754064233|
|           74100.0|117718.26869017631|
|           74700.0| 134357.9856721419|
|           90000.0| 175915.9903214057|
|          104200.0|  166488.883189396|
|           74100.0|122930.95757309394|
|           67500.0|113698.24431570992|
|          103100.0|-8421.805367283523|
|           92500.0|140800.16343181813|
|          128100.0|191174.88680318417|
|           99600.0|152620.17806110252|
+------------------+------------------+
only showing top 20 rows



In [137]:
trainingSummary = model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("MAE: %f" % trainingSummary.meanAbsoluteError)
print("r2: %f" % trainingSummary.r2)

RMSE: 68837.809284
MAE: 49825.431440
r2: 0.642521


In [138]:
trainingSummary2 = model2.summary
print("RMSE: %f" % trainingSummary2.rootMeanSquaredError)
print("MAE: %f" % trainingSummary2.meanAbsoluteError)
print("r2: %f" % trainingSummary2.r2)

RMSE: 69775.720202
MAE: 50930.726802
r2: 0.632714


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

In [139]:
spark.stop()

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Не забывай останавливать сессию `spark.stop()`</font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>👍</font>

# Общий вывод

Задача - предсказать медианную стоимость дома на основании данных о жилье в Калифорнии в 1990 году. После запуска Spark сессии были просмотрены типы данных в разных колонках, и сама таблица в целом. Далее, пустые значения NA были заменены на среднее значение в колонке total_bedrooms. Далее единственный категориальный признак был стандартизирован и кодирован с помощью StringIndexer и OHE. Также были стандартизированы количественные признаки, и объединены с категориальным значением. Далее данные были разделены на обучающую и тестовую выборки, и была применена линейная регрессия на выборках с категориальным признаком и без. В результате лучше отработала модель линейной регрессии на выборке с категориальным признаком в составе. Однако, различия в RMSE и MAE минимальны - около $1. А R2 даже выше чем в выборке с количественными признаками на 0.01. В целом по моему мнению такая ошибка в предсказании недопустима, и нужно найти более подходящую модель с меньшим показателем погрешности RMSE и MAE.

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'><s> Очень не хватает общего вывода. В выводах можно приводить полученные ранее значения. Также можно расписать все, что было сделано в ходе проведения работы.</s></font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'>Нурлан, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Поправь работу с данными(toPandas только для агрегированных данных).</font></ul>
<ul><font color='red'>Стандартизация и КОдировка для train и test по отдельности скалером обученным на train.</font></ul>
<ul><font color='red'>Добавь Общий вывод и Анализ метрик.</font></ul>

<font color='blue'>Что можно сделать лучше:</font>
<font color='orange'>В работе я оставил несколько советов. Буду рад, если ты учтешь их.</font></ul>

<font color='blue'><b>Жду новую версию проекта :)</b></font>

<font color='blue'><b>Итоговый комментарий ревьюера 2</b></font>\
<font color='green'>Нурлан, большое спасибо за доработку. В Пандас выгружаются только агрегированные данные, написан Общий вывод по работе.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Стандартизация и Кодировка для train и test по отдельности скалером обученным на train.</font></ul>

<font color='blue'><b>Жду новую версию проекта :)</b></font>

<font color='purple'><b>Комментарий студента 2</b></font> Скалер ни на капельку не улучшил модель :(

<font color='blue'><b>Комментарий ревьюера 3: </b></font> ✔️\
<font color='green'>Здесь вопрос был не в улучшении метрик, а в корректности проведённого теста. Сейчас мы устранили возможную утечку информации из трейна в тест, улучшить метрики это никак не могло, но теперь мы можем этим метрикам доверять ))).</font>

<font color='blue'><b>Итоговый комментарий ревьюера 3</b></font>\
<font color='green'> Нурлан, проект принят! \
Все этапы пройдены. Все критичные рекомендации учтены.\
Надеюсь, тебе понравился процесс выполнения и результат.</font> \
<font color='blue'><b>Спасибо, удачи в освоении профессии!</b></font>